In [1]:
from quantum_executor import QuantumExecutor

QuantumExecutor.default_providers()

['azure', 'braket', 'ionq', 'local_aer', 'qbraid', 'qiskit']

In [2]:
executor = QuantumExecutor(providers=["local_aer", "ionq", "qiskit"])

In [3]:
executor.virtual_provider.get_providers()

{'ionq': <qbraid.runtime.ionq.provider.IonQProvider at 0x318f5ba70>,
 'local_aer': <quantum_executor.local_aer.provider.LocalAERProvider at 0x318f5bdd0>,
 'qiskit': <qbraid.runtime.ibm.provider.QiskitRuntimeProvider at 0x318f5be30>}

In [4]:
executor.virtual_provider.get_backends()

{'ionq': {'qpu.forte-1': <qbraid.runtime.ionq.device.IonQDevice('qpu.forte-1')>,
  'simulator': <qbraid.runtime.ionq.device.IonQDevice('simulator')>},
 'local_aer': {'aer_simulator': <quantum_executor.local_aer.device.LocalAERBackend('aer_simulator')>,
  'fake_algiers': <quantum_executor.local_aer.device.LocalAERBackend('fake_algiers')>,
  'fake_almaden': <quantum_executor.local_aer.device.LocalAERBackend('fake_almaden')>,
  'fake_armonk': <quantum_executor.local_aer.device.LocalAERBackend('fake_armonk')>,
  'fake_athens': <quantum_executor.local_aer.device.LocalAERBackend('fake_athens')>,
  'fake_auckland': <quantum_executor.local_aer.device.LocalAERBackend('fake_auckland')>,
  'fake_belem': <quantum_executor.local_aer.device.LocalAERBackend('fake_belem')>,
  'fake_boeblingen': <quantum_executor.local_aer.device.LocalAERBackend('fake_boeblingen')>,
  'fake_bogota': <quantum_executor.local_aer.device.LocalAERBackend('fake_bogota')>,
  'fake_brisbane': <quantum_executor.local_aer.device

In [5]:
from qiskit import QuantumCircuit

# Qiskit circuit
qiskit_circuit = QuantumCircuit(2)
qiskit_circuit.h(0)
qiskit_circuit.cx(0, 1)
qiskit_circuit.measure_all()

In [6]:
openqasm_circuit = """OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg c[2];
h q[0];
cx q[0], q[1];
measure q -> c;
"""

In [7]:
dispatch = {
    "local_aer": {  # Local Aer provider
        "fake_torino": [
            {
                "circuit": qiskit_circuit,
                "shots": 1024,
                "config": {"seed": 42},
            },
            {
                "circuit": openqasm_circuit,
                "shots": 2048,
                "config": {"seed": 24},
            },
        ],
        "aer_simulator": [
            {
                "circuit": qiskit_circuit,
                "shots": 1024,
            },
        ],
    },
    "ionq": {  # IonQ provider
        "simulator": [
            {
                "circuit": qiskit_circuit,
                "shots": 1024,
                "config": {"noise": {"model": "aria-1"}},
            },
        ],
    },
}

In [8]:
results = executor.run_dispatch(dispatch, multiprocess=True, wait=True)

results

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=8dbe906f-4ae9-45d7-804c-b825c49961c9, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 500, '01': 22, '10': 5, '11': 497}), JobResult(job=Job(id=3dec625e-8fa1-4bcb-aaad-532831d377ab, circuit_type=str, shots=2048, config={}), status=Complete, data={'00': 1046, '01': 24, '10': 21, '11': 957})], 'aer_simulator': [JobResult(job=Job(id=6d1ef4ed-952c-4464-bbcd-b227db0abf19, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 533, '11': 491})]}, 'ionq': {'simulator': [JobResult(job=Job(id=cc09b634-684a-49f2-8ef9-e971cc0f4aa3, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 512, '11': 512})]}})

In [9]:
results = executor.run_dispatch(dispatch, multiprocess=True, wait=False)

results

ResultCollector(complete_jobs=0, total_jobs=4, complete=False)

In [10]:
results.complete

False

In [11]:
results.get_jobs()

{'local_aer': {'fake_torino': [JobResult(job=Job(id=fa296540-8b1e-4f3f-a68b-d846d38351fa, circuit_type=QuantumCircuit, shots=1024, config={}), status=Pending, data=None),
   JobResult(job=Job(id=8c515431-496f-4d39-a39c-402d5c8a9f19, circuit_type=str, shots=2048, config={}), status=Pending, data=None)],
  'aer_simulator': [JobResult(job=Job(id=4f34b011-d014-4c8b-b8fb-174d616c066e, circuit_type=QuantumCircuit, shots=1024, config={}), status=Pending, data=None)]},
 'ionq': {'simulator': [JobResult(job=Job(id=b082287b-ce7f-48ee-b356-0a60ee5c84f1, circuit_type=QuantumCircuit, shots=1024, config={}), status=Pending, data=None)]}}

In [12]:
results.wait_for_completion()

True

In [13]:
results.get_jobs()

{'local_aer': {'fake_torino': [JobResult(job=Job(id=fa296540-8b1e-4f3f-a68b-d846d38351fa, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 516, '01': 10, '10': 6, '11': 492}),
   JobResult(job=Job(id=8c515431-496f-4d39-a39c-402d5c8a9f19, circuit_type=str, shots=2048, config={}), status=Complete, data={'00': 1000, '01': 19, '10': 21, '11': 1008})],
  'aer_simulator': [JobResult(job=Job(id=4f34b011-d014-4c8b-b8fb-174d616c066e, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 477, '11': 547})]},
 'ionq': {'simulator': [JobResult(job=Job(id=b082287b-ce7f-48ee-b356-0a60ee5c84f1, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 512, '11': 512})]}}

In [14]:
results.get_results()

{'local_aer': {'fake_torino': [{'00': 516, '01': 10, '10': 6, '11': 492},
   {'00': 1000, '01': 19, '10': 21, '11': 1008}],
  'aer_simulator': [{'00': 477, '11': 547}]},
 'ionq': {'simulator': [{'00': 512, '11': 512}]}}

In [15]:
results.wait_for_completion()

True

In [16]:
results.get_jobs()

{'local_aer': {'fake_torino': [JobResult(job=Job(id=fa296540-8b1e-4f3f-a68b-d846d38351fa, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 516, '01': 10, '10': 6, '11': 492}),
   JobResult(job=Job(id=8c515431-496f-4d39-a39c-402d5c8a9f19, circuit_type=str, shots=2048, config={}), status=Complete, data={'00': 1000, '01': 19, '10': 21, '11': 1008})],
  'aer_simulator': [JobResult(job=Job(id=4f34b011-d014-4c8b-b8fb-174d616c066e, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 477, '11': 547})]},
 'ionq': {'simulator': [JobResult(job=Job(id=b082287b-ce7f-48ee-b356-0a60ee5c84f1, circuit_type=QuantumCircuit, shots=1024, config={}), status=Complete, data={'00': 512, '11': 512})]}}

In [17]:
results.get_results()

{'local_aer': {'fake_torino': [{'00': 516, '01': 10, '10': 6, '11': 492},
   {'00': 1000, '01': 19, '10': 21, '11': 1008}],
  'aer_simulator': [{'00': 477, '11': 547}]},
 'ionq': {'simulator': [{'00': 512, '11': 512}]}}

In [18]:
from quantum_executor import Dispatch

dispatch = Dispatch()

dispatch.add_job("local_aer", "fake_torino", qiskit_circuit, shots=1024, config={"seed": 42})
dispatch.add_job("local_aer", "fake_torino", openqasm_circuit, shots=2048, config={"seed": 24})
dispatch.add_job("local_aer", "aer_simulator", qiskit_circuit, shots=1024)
dispatch.add_job("ionq", "simulator", qiskit_circuit, shots=1024, config={"noise": {"model": "aria-1"}})

dispatch

Dispatch({'local_aer': {'fake_torino': [Job(id=bf458b67-48bd-42d7-b1f7-33d9ce169b5b, circuit_type=QuantumCircuit, shots=1024, config={'seed': 42}), Job(id=54d721d1-c344-4e42-8fc0-2d123a47ee68, circuit_type=str, shots=2048, config={'seed': 24})], 'aer_simulator': [Job(id=3857521b-6cf5-45ca-b577-0395c98423d9, circuit_type=QuantumCircuit, shots=1024, config={})]}, 'ionq': {'simulator': [Job(id=06710e4c-3e48-4949-8d7a-f088c1293697, circuit_type=QuantumCircuit, shots=1024, config={'noise': {'model': 'aria-1'}})]}})

In [19]:
from typing import Any

from quantum_executor.dispatch import Dispatch
from quantum_executor.virtual_provider import VirtualProvider


def split(
    circuit: Any,  # noqa: ANN401
    shots: int,
    backends: dict[str, list[str]],
    _virtual_provider: VirtualProvider,  # pylint: disable=unused-argument
    policy_data: Any | None = None,  # noqa: ANN401
) -> tuple[Dispatch, Any]:
    """Uniformly distribute the same circuit and shot count to every backend.

    Parameters
    ----------
    circuit : Any
        The quantum circuit to run.
    shots : int
        Number of shots for the circuit.
    backends : Dict[str, List[str]]
        A dictionary mapping provider names to lists of backend names.
    _virtual_provider : VirtualProvider
        An instance of VirtualProvider; not used in this policy.
    policy_data : Any, optional
        Additional data carried along; not used in this policy.

    Returns
    -------
    Tuple[Dispatch, Any]
        A tuple containing the Dispatch object with registered jobs and the unchanged blob.

    """
    bakends_num = sum(len(backends_ls) for backends_ls in backends.values())
    backends_shots = [shots // bakends_num] * bakends_num
    # If shots is not divisible by the number of backends, distribute the remainder
    remainder = shots % bakends_num
    for i in range(remainder):
        backends_shots[i] += 1

    dispatch = Dispatch()
    for provider_name, backends_ls in backends.items():
        for backend_name, backend_shots in zip(backends_ls, backends_shots, strict=False):
            dispatch.add_job(
                provider_name=provider_name,
                backend_name=backend_name,
                circuits=circuit.copy(),
                shots=backend_shots,
            )
    return dispatch, policy_data

In [20]:
executor.add_policy("my_split_policy", split)
# executor.add_policy_from_file("my_split_policy_file.py")

In [21]:
backends = {
    "local_aer": ["fake_torino", "aer_simulator"],
    "ionq": ["simulator"],
}

results = executor.run_experiment(
    circuit=qiskit_circuit,
    shots=1024,
    backends=backends,
    split_policy="my_split_policy",
    multiprocess=True,
    wait=True,
)

results

ResultCollector({'local_aer': {'fake_torino': [JobResult(job=Job(id=04c58938-03a7-41bd-a591-7952f7c2ae41, circuit_type=QuantumCircuit, shots=342, config={}), status=Complete, data={'00': 169, '01': 6, '10': 3, '11': 164})], 'aer_simulator': [JobResult(job=Job(id=b26f3542-3158-4cf5-bf58-1b8e06012d92, circuit_type=QuantumCircuit, shots=341, config={}), status=Complete, data={'00': 175, '11': 166})]}, 'ionq': {'simulator': [JobResult(job=Job(id=19508556-ac42-429a-946b-7ffdfb9f87ca, circuit_type=QuantumCircuit, shots=342, config={}), status=Complete, data={'00': 171, '11': 171})]}})

In [ ]:
from quantum_executor.job_runner import ResultData


def merge(
    results: dict[str, dict[str, list[ResultData]]],
    policy_data: Any,  # noqa: ANN401
) -> tuple[dict[str, int], Any]:
    """Merge job results by summing bitstring counts.

    Parameters
    ----------
    results : Dict[str, Dict[str, List[ResultData]]]
        Nested mapping (provider -> backend -> list of ResultData objects).
    policy_data : Any
        Additional data carried along; not used in this policy.

    Returns
    -------
    Tuple[Dict, Any]
        A tuple containing the merged counts dictionary and the unchanged blob.

    """
    merged_results: dict[str, int] = {}
    for _, provider_results in results.items():  # pylint: disable=too-many-nested-blocks
        for _, job_results in provider_results.items():
            for result_data in job_results:
                if result_data is not None and isinstance(result_data, dict):
                    for bitstring, count in result_data.items():
                        if isinstance(count, int):
                            merged_results[bitstring] = merged_results.get(bitstring, 0) + count
    return merged_results, policy_data

In [23]:
executor.add_policy("my_merge_policy", merge_policy=merge)
# executor.add_policy_from_file("my_merge_policy_file.py")

In [24]:
results = executor.run_dispatch(dispatch, multiprocess=True, wait=True, merge_policy="my_merge_policy")

# results = executor.run_experiment(
#     circuit=qiskit_circuit,
#     shots=1024,
#     backends=backends,
#     split_policy="my_split_policy",
#     merge_policy="my_merge_policy",
#     multiprocess=True,
#     wait=True,
# )

results

MergedResultCollector(merged_results={'00': 2592, '01': 32, '10': 41, '11': 2455}, initial_policy_data={}, final_policy_data={})